# Import libraries and data

In [1]:
import os
import openai
import pandas as pd
import numpy as np
import openpyxl
import chardet

In [2]:
# Get OpenAI API secret key
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Get annotated tweets
with open('../data/local/paho_tweets_filtered.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


paho = pd.read_csv('../data/local/paho_tweets_filtered.csv', encoding=result['encoding']).reset_index().iloc[:, 2:]

paho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       849 non-null    object 
 1   positive   849 non-null    int64  
 2   neutral    849 non-null    int64  
 3   negative   849 non-null    int64  
 4   total      849 non-null    int64  
 5   neg_per    849 non-null    float64
 6   pos_per    849 non-null    float64
 7   neu_per    849 non-null    float64
 8   agreement  849 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 59.8+ KB


In [4]:
paho.shape[0]

849

# Using sample to test prompt

In [5]:
# Prompt for GPT
prompt_raw_1 = "Qual é a atitude do autor em relação às vacinas expresso pelo usuário do texto do tweet delimitado por crases triplos? \
Use uma das seguintes palavras: neutro, negativo, positivo. Inclua uma explicação para a seleção do sentimento. \
```{TEXTO_DO_TWEET_AQUI}```"

In [6]:
# Prompt for GPT
prompt_raw_2 = "Qual é a atitude do autor em relação às vacinas expresso pelo usuário do texto do tweet delimitado por crases triplos? \
Use uma das seguintes palavras: neutro, negativo, positivo. Inclua uma explicação para a seleção do sentimento. \
\
Exemplos de sentimento positivo: \
* Por favor não se esqueçam de tomar a vacina! \
* Vacina contra o câncer da pele é testada com sucesso. \
\
Examplos de sentimento neutro: \
* Fui tomar vacina ontem. \
* A nova campanha de vacinação contra a gripe começou hoje. \
\
Examplos de sentimento negativo: \
* Tomei vacina contra a gripe e meu braço ta doendo. \
* Estudo mostra efeitos colaterais sérios da nova vacina contra a gripe. \
\
```{TEXTO_DO_TWEET_AQUI}```"

In [7]:
# Prompt for GPT
prompt_raw_3 = "Give me the sentiment regarding vaccination expressed by the user of the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [8]:
len(paho.index)

849

In [9]:
paho.iloc[0,0]

'"@<user>: Governo inclui no #SUS nova vacina contra coqueluche para grávidas e recém-nascidos: <url>'

In [10]:
id_gpt = []
sent_gpt = []
text = []

In [14]:
for i in range(588, len(paho.index)-1):
    prompt_i = prompt_raw_3.replace('TWEET_TEXT_HERE', paho.iloc[i,0])
    response = openai.ChatCompletion.create(
      #model="text-davinci-003",
      model = "gpt-4",
      messages=[{"role": "user", "content": prompt_i}],
      temperature=0.8,
      #max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    print(i)
    text.append(paho.iloc[i,0])
    sent_gpt.append(response.choices[0].message.content)

588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837


In [63]:
for i in range(0, 422):
    prompt_i = prompt_raw_2.replace('TEXTO_DO_TWEET_AQUI', paho.iloc[i,0])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    text.append(paho.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [15]:
df_gpt_total = pd.DataFrame(list(zip(text, sent_gpt)),
                            columns = ['text', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           848 non-null    object
 1   sentiment_gpt  848 non-null    object
dtypes: object(2)
memory usage: 13.4+ KB


In [16]:
df_gpt_total['prompt'] = 3

In [17]:
df_gpt_total['model'] = "GPT4"

In [ ]:
df_gpt_total.tail(10)

In [18]:
len(df_gpt_total)

848

In [19]:
df_gpt_total_all = df_gpt_total

In [37]:
#df_gpt_total_all = df_gpt_total_all.append(df_gpt_total)

In [20]:
len(df_gpt_total_all)

848

In [ ]:
df_gpt_total_all.tail()

In [21]:
df_gpt_total_all.to_csv('../data/local/gpt_sentiment_paho_prompt3_gpt4.csv')

# Using three categories for vaccine sentiment

In [10]:
# Prompt for GPT
prompt_raw = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. \
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [11]:
len(tweets.index)

4786

In [12]:
test = tweets.iloc[:101, :]
test.info

<bound method DataFrame.info of      index                                               text  neutral  \
0        0  "#Shingrix Vaccine Against Shingles Remains In...        3   
1        1  "...French scientist Louis Pasteur coined the ...        3   
2        2  "...due to potentially unexplained illnesses."...        2   
3        3  "...on average, it takes 10.7 years to develop...        3   
4        4  "..only 29 percent of the teens its members in...        3   
..     ...                                                ...      ...   
96      96  #COVID19 vaccine could be delivered in mailabl...        4   
97      97  #COVID19Vaccine in 1 month? CSIR pins hope on ...        4   
98      98  #COVID19dz Ignorance at the utmost level as co...        1   
99      99  #CRO Vaccine Market in Austria to 2021 Market ...        3   
100    100  #California has just implemented one of the #s...        3   

     negative  positive  total  
0           0         0      3  
1           0

In [7]:
tweets1 = tweets.iloc[0:306,:]
tweets2 = tweets.iloc[306:601,:]
tweets3 = tweets.iloc[601:901, :]
tweets4 = tweets.iloc[901:1201, :]


In [14]:
test.iloc[1,1]

'"...French scientist Louis Pasteur coined the term “vaccination” in honour of Jenner’s work (vacca being the Latin for cow)" - TIL! 🐮💉 <url>'

In [68]:
id_gpt = []
sent_gpt = []

In [69]:
for i in range(0,4785):
    prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[i,1])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [38]:
run = np.array([1])

In [70]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4785 entries, 0 to 4784
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         4785 non-null   int64 
 1   sentiment_gpt  4785 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.9+ KB


In [71]:
df_gpt_total['run'] = 3

In [72]:
df_gpt_total.tail()

,id_gpt,sentiment_gpt,run
4780,4780,\n\nNegative,3
4781,4781,\n\npositive,3
4782,4782,\n\nneutral,3
4783,4783,\n\npositive,3
4784,4784,\n\nNeutral,3


In [59]:
#df_gpt_total_all = df_gpt_total.append(df_gpt_total2)

C:\Users\lespinosa\AppData\Local\Temp\ipykernel_23560\3678373193.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gpt_total_all = df_gpt_total.append(df_gpt_total2)


In [73]:
df_gpt_total.to_excel('../data/local/gpt_sentiment_mturk_v4.xlsx')


# Using two categories for vaccine sentiment

In [6]:
tweets_pos_neg = tweets[tweets['label_tag'].isin(['positive', 'negative'])]
tweets_pos_neg.head()

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive
26,26,1237492335430250496,@user @user thank you for the vaccine LOL #Wel...,positive
28,28,1237894709306986496,Remember polio? Killed a lot of people. Vaccin...,positive
38,38,1239077015250030592,You be ready to pay what this Coronavirus vacc...,negative


In [39]:
tweets_new_pos_neg = tweets_new[tweets_new['label_tag'].isin(['positive', 'negative'])]
tweets_new_pos_neg.head()

,index,id,text,label_tag
1,1,1008751928568840192,"Great news to see. Hopefully, folks who disput...",positive
2,2,1008751948957351936,The global community must invest in further re...,positive
3,3,1008752099222523904,HPV vaccine has almost wiped out infections in...,positive
4,4,1010156916553736192,So I can catch this shit too and need treatmen...,negative
5,5,1010157450618601472,Could a #vaccine for #type-1 diabetes be on th...,positive


In [42]:
prompt_raw_pos_neg = "Please give me the sentiment regarding vaccination in the following tweet: \
"TWEET_TEXT_HERE\"\nUse one of the following words: negative, positive"

In [40]:
len(tweets_new_pos_neg.index)

128

In [9]:
test = tweets_pos_neg.iloc[0:2, :]
test

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive


In [7]:
tweets_pos_neg1 = tweets_pos_neg.iloc[0:306,:]
tweets_pos_neg2 = tweets_pos_neg.iloc[306:601,:]
tweets_pos_neg3 = tweets_pos_neg.iloc[601:901, :]
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]


In [103]:
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]

In [43]:
id_gpt_two = []
sent_gpt_two = []

In [44]:
for i in range(0,128):
    prompt_two_i = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_new_pos_neg.iloc[i,2])
    response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_i,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt_two.append(tweets_new_pos_neg.iloc[i,0])
    sent_gpt_two.append(response_two.choices[0].text)

In [17]:
prompt_two_1339 = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_pos_neg.iloc[1339,2])
response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_1339,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
id_gpt_two.append(tweets_pos_neg.iloc[1339,0])
sent_gpt_two.append(response_two.choices[0].text)

In [46]:
df_two_gpt_total = pd.DataFrame(list(zip(id_gpt_two, sent_gpt_two)),
                            columns = ['id_gpt_two', 'sentiment_gpt_two'])
df_two_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_gpt_two         128 non-null    int64 
 1   sentiment_gpt_two  128 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [47]:
df_two_gpt_total.tail()

,id_gpt_two,sentiment_gpt_two
123,179,", or neutral\n\nPositive"
124,180,", or neutral\n\nPositive"
125,181,", or neutral\n\nNegative"
126,182,", or neutral\n\nPositive"
127,184,\n\nNegative


In [108]:
id_gpt13 = id_gpt
sent_gpt13 = sent_gpt
df_gpt_total13 = df_gpt_total

In [25]:
#df_gpt_total1.to_csv('../data/local/gpt_sentiment1.csv')
df_gpt_total1['sentiment_gpt2'] = df_gpt_total1['sentiment_gpt'].to_string()
print(type(df_gpt_total1['sentiment_gpt2']))

<class 'pandas.core.series.Series'>


In [30]:
#sent_gpt1.to_csv('../data/local/gpt_sentiment1_only.csv')

In [48]:
df_two_gpt_total.to_excel('../data/local/gpt_sentiment_new_pos_neg.xlsx')
